# Overture Admin Areas Tutorial

The [Overture Maps Foundation](https://overturemaps.org) recently published its first-release version of open map data 2023-07-26-alpha.0. Included in this is the Administrative Boundaries Theme, which contains the geometries of Administrative Areas down to the state level (admin_level 2 & 4). Administrative area polygons form the basis for what we often consider to be a "world map" and contribute to effective spatial representation, analysis, and decision-making.

Administrative areas are used as the backbone for demographic profiling, service accessibility assessments and determining where important rescources are located and should be allocated. Visualizing administrative areas on a digital map makes complex demographic divisions more understandable to a wide audience. This can aid communication and decision-making for governance, economics, public health, urban planning, environmental management, and more.

In this tutorial, we will look at how to:

1. Read Admin data from Overture Maps.
2. Make spatial queries on the admin data.
3. Combine the data with external open source data.
4. Display the geospatial result for anlysis. 

## Read Admin data from Overture Maps

Overture Maps data is stored in Azure blob storage and AWS S3 buckets using parkets. Details on the map schema and where to get the official data can be found [here](https://overturemaps.org/download/).

To read this data, we will use duckdb to read the parquet files. This uses the example query provided [here](https://github.com/OvertureMaps/data/blob/main/README.md#3-duckdb-sql)

Note: The full dataset is `~1.2Gb`. It only needs to be downloaded once so make sure to only run this secion of the tutorial for the initial download step.

In [ ]:
import duckdb
db = duckdb.connect()
db.execute("INSTALL spatial")
db.execute("INSTALL httpfs")
db.execute("""
LOAD spatial;
LOAD httpfs;
SET s3_region='us-west-2';
""")

In [ ]:
db.execute("""
COPY (
    SELECT
           type,
           subType,
           localityType,
           adminLevel,
           isoCountryCodeAlpha2,
           JSON(names) AS names,
           JSON(sources) AS sources,
           ST_GeomFromWkb(geometry) AS geometry
      FROM read_parquet('s3://overturemaps-us-west-2/release/2023-07-26-alpha.0/theme=admins/type=*/*', filename=true, hive_partitioning=1)
     WHERE adminLevel in (2, 4)
       AND ST_GeometryType(ST_GeomFromWkb(geometry)) IN ('POLYGON','MULTIPOLYGON')
) TO 'data/admin.geojson'
WITH (FORMAT GDAL, DRIVER 'GeoJSON');
""")

### Convert data into a usebale format using Geopandas

With the data downloaded, we can use geopandas to read the file using `.read_file()`. This takes a string path to the file location, and reads it into a geodataframe.

In [ ]:
import geopandas as gpd
import pandas as pd

In [ ]:
# Filepath
fp = "data/admin.geojson"

# Read the file into a dataframe
admin_data_df = gpd.read_file(fp)

# Take a look at the data
admin_data_df.head()

In [ ]:
shape = admin_data_df.shape
print("This dataframe has [{}] rows and [{}] columns".format(shape[0], shape[1]))

Now that we have our data in a dataframe, we can use pandas and geopandas API's to query and manipulate it into differnt shapes that we would like.

Let's practice this by extracting the local and english names of all the features in the `admin_data_df` dataframe, and adding them as a new column to the dataframe.

Names for Overture admin features are added by TomTom, and enriched using ESRI datasets. To make the data useable in as many countries as possible, these names are stored as a JSON object in the data to allow users the flexibilty to select the names they would like to use for thier application. The `local` name is the name of the feauter in the official language of the country (where applicable) and can be considered to be the features "defualt" name, as recognised by the citizens of that country.

To extract and append these names to the dataframe, we will use a function that reads the names JSON column, and extracts the values where language is `local` and `en` into two new columns `localname` and `enname`. This function will then be apllied to each row of the dataframe usineg the `.apply()` pandas function, with the `axis=1` input parameter to specify that we want this function to be applied row-wise.

In [ ]:
def append_local_and_en_names(row):
    # read the names JSON list if available
    names = row['names']['common'] if pd.notna(row['names']) else []

    for name in names:
        # append local name
        if name['language'] == 'local':
            row['localname'] = name['value']
        
        # append english name
        if name['language'] == 'en' or name['language'] == 'en-Latn':
            row['enname'] = name['value']
    
    return row

admin_data_df = admin_data_df.apply(append_local_and_en_names, axis=1)

admin_data_df.head()

In [ ]:
shape = admin_data_df.shape
print("This dataframe has [{}] rows and [{}] columns".format(shape[0], shape[1]))

Now that we have our new name columns, let's create two new subsets of our data, using only the colums we need. 

Here, we will use the pandas `.query()` function to select the rows where the `adminlevel` value is `2` for country, or `4` for states. These two dataframes will be used later to perform spatial queries. We will make this query on the columns we need from `admin_data_df`, leaving us with a dataframe that has only the columns and rows that satisfy these contions for each subset.


In [ ]:
countries_df = admin_data_df[['localname','enname','localitytype','isocountrycodealpha2','adminlevel','geometry']].query("adminlevel == 2")
countries_df.head()

In [ ]:
shape = countries_df.shape
print("This dataframe has [{}] rows and [{}] columns".format(shape[0], shape[1]))

In [ ]:

states_df = admin_data_df[['localname','enname','localitytype','isocountrycodealpha2','adminlevel','geometry']].query("adminlevel == 4")
states_df.head()

In [ ]:
shape = states_df.shape
print("This dataframe has [{}] rows and [{}] columns".format(shape[0], shape[1]))

With these two subsets created, we can go ahead to view our data on a map! Let's start with `countries_df`. We expect to see a map of the planet that shows all the countries in the world. To do this, we will use the geopandas `.explore()` method to display our data.


In [ ]:
countries_df.crs


NOTE: This is where we start coming up against some of the challenges of using geospatial data. Due to the sheer size of some of the features (countries are roughly 140Mb, states are ~1Gb) rendering this data can be fairly memory intensive. 
We can overcome this by managing the resolution of features using the geopandas `.simplify()` method. This takes a tollerace value in the units of the [Coordinate Reference Systems](https://docs.qgis.org/3.4/en/docs/gentle_gis_introduction/coordinate_reference_systems.html) (CRS) used by the geospatial data. Here, we use the `WGS 84` system, which uses `degree` as its unit of measurement.

What does this all mean?

If you would like to load the data for visual inspection, while keeping a good balance between your local rescources and the accuracy of the displayed data, you can set the resolution using the following guide.

- Low resolution    : Easiest to load       -> `tolerance=0.1`
- Medium resolution : A good balance        -> `tolerance=0.01`
- High resolution   : Farily accurate       -> `tolerance=0.001`
- Full resolution   : Exact representation  -> use `.explore()` directly.

In [ ]:
def display_simplified_df(df, column, tolerance, cmap='Set1'):
    # Copy the data to a new df only for visualisation
    simp_df = df.copy()

    # Simplify the geometry, and reset the column to display
    simp_df['simplegeom'] = simp_df.simplify(tolerance=tolerance)
    simp_df.set_geometry('simplegeom', inplace=True)

    # Display the data, and colour based on the specified column
    simp_df.set_geometry('simplegeom', inplace=True)
    return simp_df.explore(column=column, cmap=cmap)

# Display country geometries.
display_simplified_df(countries_df, 'isocountrycodealpha2', 0.01)

## Make spatial queries on the admin data.

We have been able to read, manipulate and view our geospatial data. Now, we will go over how to make spatial queries to find the states of a given location. 

To do this, we will use a custom method that uses the geopandas `.sjoin()` method to find the states of a specified country, using the `countries_df` and `states_df` dataframes. We use the `within` predicate to search only for geometries that are contained within the given country geometry.

In [ ]:
def get_country_states(c_df, s_df, isoCode):
    # find the specific country
    country = c_df[c_df['isocountrycodealpha2'] == isoCode] 
    # Find the states in that country
    states = s_df.sjoin(country[['isocountrycodealpha2','geometry']], predicate="within")
    # Drop uneeded columns
    states.drop(columns=['isocountrycodealpha2_right','isocountrycodealpha2_left','index_right'], inplace=True)
    return states

We will test this function by searching for the states in Ghana, using its ISO:alpha2 country-code `GH`.

Note: We are using `.explore()` directly here because the scope of these geometries is relatively small, making a higher accuracy more desirable. However, if this operation is tasking on your development environment, change this to use the `display_simplified_df()` function.

In [ ]:
gh_states_df = get_country_states(c_df=countries_df, s_df=states_df, isoCode='GH')
gh_states_df.explore('localname', cmap='Set1')

## Combine the data with external open source data.

Now that we have been able to read Overture maps data, display it and perform spatial queries on it, we can start using it with other available open source or proprierary data. 

In this example, we will use open source data from [Stats Bank Ghana](https://statsbank.statsghana.gov.gh) to try and answer a simple research question: Where are we most likely to find people who can use Overture Maps data.

To answer this, we will use their "Use of Internet on Laptop in Last 3 months of Population (6 years and older)" [survey](https://statsbank.statsghana.gov.gh/pxweb/en/PHC%202021%20StatsBank/PHC%202021%20StatsBank__ICT/use_internet_on_device_3.px/). This survety contains the counts of people by Education, Locality, Geographic_Area, Sex and Age, who have used the internet on a laptop in the last 3 months. This should give us a rough estimate of how many poeple we may be able to find in a given region that can use this data.

To do this, we will read in the data as a CSV, and store it in a pandas dataframe to simplify its usage.

In [ ]:
fp = 'data/ICT17.csv'
ict17_df = pd.read_csv(fp)
ict17_df.head()

In [ ]:
shape = ict17_df.shape
print("This dataframe has [{}] rows and [{}] columns".format(shape[0], shape[1]))

As we can see, this is a very comprehensive dataset, with each category combination counted on a separate row. 

To use this, we will need to create a subset of the data with only the criteria we want to display. Let's start with the counts of people between the ages of 15 and 29 who have used the internet on a laptop in the last 3 months, and live in urban areas.

In [ ]:
young_urban_users_df = ict17_df[['Used_laptop', 'Education', 'Locality','Geographic_Area', 'Sex', '15-19', '20-24','25-29']].query("Used_laptop == 'Yes' and Locality == 'Urban' and Sex == 'Both sexes' and Education == 'Total'")
young_urban_users_df.head()


In [ ]:
shape = young_urban_users_df.shape
print("This dataframe has [{}] rows and [{}] columns".format(shape[0], shape[1]))

We can further improve this data by creating a new column for the total count of people ages 15-29, by using the `.sum()` pandas method on our three numerical columns, and adding up by row using `axis=1`.

In [ ]:
young_urban_users_df['15-29'] = young_urban_users_df[['15-19', '20-24','25-29']].sum(axis=1)
young_urban_users_df.head()

With this new dataframe, we can merge our geospatial data using the pandas `.merge()` function. This joins our two datasets on a shared column value to give us a combined dataset with the columns from both our geospatial, and survey datasets. We do this by setting `left_on='localname', right_on='Geographic_Area'`, which means that the data will be joined using the state names present in both of these columns from our left (geospatial) dataframe, and our right (survey) dataframe. 

Finally, the `how='left'` parameter is also set to ensure that only all the names present in our geospatial dataframe are used.

In [ ]:
gh_states_and_data_df = pd.merge(left=gh_states_df, right=young_urban_users_df[['Geographic_Area', 'Locality', '15-29']], how='left', left_on='localname', right_on='Geographic_Area')
gh_states_and_data_gdf = gpd.GeoDataFrame(gh_states_and_data_df)
gh_states_and_data_gdf.head()

The combined data can now be displayed using the `.explore()` method. Here, we use the `BuPu` colour map on the `15-29` column, to give us a good contrast between low and high numbers, making it easier to spot the states we should be focusing on for this search.


In [ ]:
gh_states_and_data_gdf.explore(column='15-29', cmap='BuPu')

As can be seen from this map, the two states we are most likely to find people that meet the search criteria are `Greater Accra` followed by `Ashanti`.

We can also observe a grey hole in the `North East`. Given that we used all the names from our geospatial data, we can inffer that this is either not the exact name used for that region, or the survey did not capture that region. It could also be that that region was captured as a group of smaller regions that may fall in the Admin Level 6-8 levels. This will rewquire further data validation. For now though, we can use what we have to start making broad assessments.

Let us tweak our search now to look for a differnt group. For examlpe, if we wanted to provide resources and support for women under the age of 25, living in rural areas, where would we be most likely to find them?

Use the following section to refine your search criteria, and see what observations you can make with the available data.

In [ ]:
# Create a search criteria subset
young_rural_female_users_df = ict17_df[['Used_laptop', 'Education', 'Locality','Geographic_Area', 'Sex', '15-19', '20-24']].query("Used_laptop == 'Yes' and Locality == 'Rural' and Sex == 'Female' and Education == 'Total'")
young_rural_female_users_df['15-24'] = young_rural_female_users_df[['15-19', '20-24']].sum(axis=1)

# Merge the subset with the geospatial data
gh_rural_female_users_by_state_df = pd.merge(left=gh_states_df, right=young_rural_female_users_df[['Geographic_Area', 'Locality', '15-24']], how='left', left_on='localname', right_on='Geographic_Area')
gh_rural_female_users_by_state_gdf = gpd.GeoDataFrame(gh_rural_female_users_by_state_df)

# Inspect the data
gh_rural_female_users_by_state_gdf.head()


In [ ]:
# Plot the data
gh_rural_female_users_by_state_gdf.explore(column='15-24', cmap='BuPu')